# CDC testing data

* [COVID-19 State and County Policy Orders](https://healthdata.gov/dataset/covid-19-state-and-county-policy-orders)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re